In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from tqdm import tqdm

In [2]:
print(tf.__version__)

0.12.0


In [3]:
tf.set_random_seed(0)

In [4]:
from data import get_data

In [5]:
X_train, y_train, X_test, y_test = get_data(one_hot=True)

In [6]:
mean_of_train = np.mean(X_train)
std_of_train = np.std(X_train)
print(mean_of_train, std_of_train)

115.481949803 65.2925375088


In [7]:
X_train = (X_train - mean_of_train) / std_of_train
X_test = (X_test - mean_of_train) / std_of_train

In [8]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(20000, 128, 128, 3) (20000, 2) (5000, 128, 128, 3) (5000, 2)


In [9]:
train_len, width, height, channel = X_train.shape
test_len = X_test.shape[0]
print(train_len, width, height, channel, test_len)

20000 128 128 3 5000


In [28]:
# 学习率
learning_rate = 0.001
# 迭代次数（批次）
n_epoch = 10
# 批次大小
# 可能需要调小
batch_size = 32

In [11]:
# 输入大小
input_size = (width, height, channel)
# 输出大小
target_size = 2

In [12]:
# 输入占位符
X = tf.placeholder(tf.float32, [batch_size, width, height, channel])
# 输出占位符
y = tf.placeholder(tf.float32, [batch_size, target_size])

In [13]:
def conv(input_layer, output_size, pitch_shape, name, strides=[1, 1, 1, 1], padding='VALID'):
    with tf.variable_scope(name):
        shape = [
            pitch_shape[0],
            pitch_shape[1],
            int(input_layer.get_shape()[-1]),
            output_size
        ]
        pitch = tf.Variable(tf.random_normal(shape))
        bias = tf.Variable(tf.zeros([output_size]))
        conv = tf.nn.bias_add(
            tf.nn.conv2d(
                input_layer, pitch, strides=strides, padding=padding
            ),
            bias
        )
        print(name, conv.get_shape())
        return conv

In [14]:
def max_pool(input_layer, name, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID'):
    mp = tf.nn.max_pool(input_layer, ksize=ksize, strides=strides, padding=padding)
    print(name, mp.get_shape())
    return mp

In [15]:
model = X

In [16]:
model = tf.nn.relu(conv(model, 64, (3, 3), 'conv_1', strides=(1, 2, 2, 1)))

conv_1 (32, 63, 63, 64)


In [17]:
model = tf.nn.relu(conv(model, 64, (3, 3), 'conv_2'))

conv_2 (32, 61, 61, 64)


In [18]:
model = max_pool(model, 'max_pool_1')

max_pool_1 (32, 30, 30, 64)


In [19]:
model = tf.nn.relu(conv(model, 128, (3, 3), 'conv_3', strides=(1, 2, 2, 1)))

conv_3 (32, 14, 14, 128)


In [20]:
model = tf.nn.relu(conv(model, 128, (3, 3), 'conv_4'))

conv_4 (32, 12, 12, 128)


In [21]:
model = max_pool(model, 'max_pool_2')

max_pool_2 (32, 6, 6, 128)


In [22]:
model = tf.nn.relu(conv(model, 256, (3, 3), 'conv_5'))

conv_5 (32, 4, 4, 256)


In [23]:
model = tf.nn.relu(conv(model, 256, (3, 3), 'conv_6'))

conv_6 (32, 2, 2, 256)


In [24]:
model = max_pool(model, 'max_pool_3')

max_pool_3 (32, 1, 1, 256)


In [25]:
model = tf.reshape(model, [batch_size, -1])

In [26]:
print(model.get_shape())
dim = int(model.get_shape()[1])
print('dim is {}'.format(dim))

(32, 256)
dim is 256


In [27]:
def full_connect(input_layer, output_size, name):
    with tf.variable_scope(name):
        weight = tf.Variable(tf.random_normal([int(input_layer.get_shape()[1]), output_size]))
        bias = tf.Variable(tf.random_normal([output_size]))
        fc = tf.add(
            tf.matmul(input_layer, weight),
            bias
        )
        print(name, fc.get_shape())
        return fc

In [29]:
model = full_connect(model, 128, 'fc_1')

fc_1 (32, 128)


In [30]:
model = full_connect(model, target_size, 'fc_2')

fc_2 (32, 2)


In [31]:
pred = model

In [32]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        pred, y
    )
)

In [33]:
opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [34]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [35]:
init = tf.global_variables_initializer()

In [36]:
def batch_flow(inputs, targets, batch_size):
    """流动数据流"""
    flowed = 0
    total = len(inputs)
    while True:
        X_ret = []
        y_ret = []
        for i in range(total):
            X_ret.append(inputs[i])
            y_ret.append([targets[i]])
            if len(X_ret) == batch_size:
                flowed += batch_size
                X, y = np.array(X_ret), np.array(y_ret)
                y = y.reshape([batch_size, -1])
                yield X, y
                X_ret = []
                y_ret = []
            if flowed >= total:
                break
        if flowed >= total:
            break

In [37]:
for batch_x, batch_y in batch_flow(X_train, y_train, batch_size):
    print(batch_x.shape, batch_y.shape)
    break

(32, 128, 128, 3) (32, 2)


In [38]:
with tf.Session() as sess:
    sess.run(init)
    total = None
    for epoch in range(n_epoch):
        costs = []
        accs = []
        for batch_x, batch_y in tqdm(batch_flow(X_train, y_train, batch_size), total=total):
            sess.run(opt, feed_dict={X: batch_x, y: batch_y})
            c, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y})
            costs.append(c)
            accs.append(acc)
        print('epoch: {}, loss: {:.4f}, acc: {:.4f}'.format(epoch, np.mean(costs), np.mean(accs)))
        if total is None:
            total = len(costs)
    print('calculate test accuracy')
    costs = []
    accs = []
    for batch_x, batch_y in tqdm(batch_flow(X_test, y_test, batch_size)):
        sess.run(opt, feed_dict={X: batch_x, y: batch_y})
        c, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y})
        costs.append(c)
        accs.append(acc)
    print('test loss: {:.4f}, acc: {:.4f}'.format(np.mean(costs), np.mean(accs)))
    print('Done')

625it [00:24, 25.66it/s]0:00,  1.35it/s]
  0%|          | 3/625 [00:00<00:28, 21.63it/s]

epoch: 0, loss: 293604192.0000, acc: 0.5859


  0%|          | 3/625 [00:00<00:22, 27.71it/s]

epoch: 1, loss: 82572320.0000, acc: 0.6215


  0%|          | 3/625 [00:00<00:21, 28.49it/s]

...

epoch: 8, loss: 3198275.0000, acc: 0.7648


100%|██████████| 625/625 [00:22<00:00, 27.52it/s]1%|▏         | 9/625 [00:00<00:22, 27.89it/s]
3it [00:00, 28.24it/s]

epoch: 9, loss: 2099150.7500, acc: 0.7854
calculate test accuracy


157it [00:05, 27.63it/s]t [00:00, 27.89it/s]

test loss: 4672728.5000, acc: 0.6242
Done


懒得调……